In [ ]:
import os
import pandas as pd
from clickhouse_driver import Client

ck_host = os.environ.get('CK_HOST') or '127.0.0.1'

client = Client(host=ck_host, port=9000, user='wuhao', password='123456')



In [ ]:
sql = """select sum(if(behavior_type = 'pv', 1, 0)) as pv, uniqExact(user_id) as uv
from test.user_behavior"""
df=client.query_dataframe(sql)
df

In [ ]:
sql="""select DATE(timestamp) as day,
       sum(if(behavior_type = 'pv', 1, 0)) as pv,
       uniqExact(user_id) as uv
from test.user_behavior
group by DATE(timestamp)
order by day"""
df=client.query_dataframe(sql)
df

###  每个用户的购物情况，加工到 user_behavior_count

In [ ]:
sql="""drop table if exists test.user_behavior_count"""
client.execute(sql)
sql="""create view test.user_behavior_count as
(select user_id,
       sum(if(behavior_type='pv',1,0)) as pv,
       sum(if(behavior_type='fav',1,0)) as fav,
       sum(if(behavior_type='cart',1,0)) as cart,
       sum(if(behavior_type='buy',1,0)) as buy
       from test.user_behavior
group by user_id)"""
client.execute(sql)

### 复购率：产生两次或两次以上购买的用户占购买用户的比例

In [ ]:
sql="""select round(sum(if(buy > 1, 1, 0)) *100 / sum(if(buy > 0, 1, 0)),2)  as p
from test.user_behavior_count"""
df=client.query_dataframe(sql)
df

### 用户行为转换率

In [ ]:
sql="""select a.pv,
       a.fav,
       a.cart,
       a.fav + a.cart                     `fav+cart`,
       a.buy,
       round((a.fav + a.cart) / a.pv, 4)  pv2favcart,
       round(a.buy / (a.fav + a.cart), 4) favcart2buy,
       round(a.buy / a.pv, 4)             pv2buy
from (select sum(pv)   pv,
             sum(fav)  fav,
             sum(cart) cart,
             sum(buy)  buy
      from test.user_behavior_count) a"""
client.query_dataframe(sql)

### 用户行为习惯

In [ ]:
sql="""select HOUR(timestamp)                  hour,
       sumIf(1, behavior_type = 'pv')   pv,
       sumIf(1, behavior_type = 'fav')  fav,
       sumIf(1, behavior_type = 'cart') cart,
       sumIf(1, behavior_type = 'buy') buy
from test.user_behavior
group by HOUR(timestamp)
order by hour"""
client.query_dataframe(sql)

### 一周用户的活跃分布

In [ ]:
sql="""select  toDayOfWeek(timestamp) weekday,
       sumIf(1,behavior_type='pv') pv,
       sumIf(1,behavior_type='fav') fav,
       sumIf(1,behavior_type='cart') cart,
       sumIf(1,behavior_type='buy') buy
from test.user_behavior
where DATE(timestamp) between '2017-11-27' and '2017-12-03'
group by weekday
order by weekday"""
client.query_dataframe(sql)

### 最近一次购买

In [ ]:
sql = """set allow_experimental_window_functions=1"""
client.execute(sql)
sql="""select user_id,
       dateDiff('day',max(timestamp),DATE('2017-12-04')) R,
       dense_rank() over( order by dateDiff('day',max(timestamp),DATE('2017-12-04'))) R_rank
from test.user_behavior
where behavior_type = 'buy'
group by user_id
limit 100;"""
client.query_dataframe(sql)

### 消费频率

In [ ]:
sql = """set allow_experimental_window_functions=1;"""
client.execute(sql)
sql="""select user_id,
       count() as F,
       dense_rank() over(order by count() desc) as F_rank
from test.user_behavior
where behavior_type = 'buy'
group by user_id
limit 10;"""
client.query_dataframe(sql)

In [ ]:
import os
import pandas as pd
from clickhouse_driver import Client

ck_host = os.environ.get('CK_HOST') or '127.0.0.1'

client = Client(host=ck_host, port=9000, user='wuhao', password='123456')



In [ ]:
sql = """select sum(if(behavior_type = 'pv', 1, 0)) as pv, uniqExact(user_id) as uv
from test.user_behavior"""
df=client.query_dataframe(sql)
df

In [ ]:
sql="""select DATE(timestamp) as day,
       sum(if(behavior_type = 'pv', 1, 0)) as pv,
       uniqExact(user_id) as uv
from test.user_behavior
group by DATE(timestamp)
order by day"""
df=client.query_dataframe(sql)
df

###  每个用户的购物情况，加工到 user_behavior_count

In [ ]:
sql="""drop table if exists test.user_behavior_count"""
client.execute(sql)
sql="""create view test.user_behavior_count as
(select user_id,
       sum(if(behavior_type='pv',1,0)) as pv,
       sum(if(behavior_type='fav',1,0)) as fav,
       sum(if(behavior_type='cart',1,0)) as cart,
       sum(if(behavior_type='buy',1,0)) as buy
       from test.user_behavior
group by user_id)"""
client.execute(sql)

### 复购率：产生两次或两次以上购买的用户占购买用户的比例

In [ ]:
sql="""select round(sum(if(buy > 1, 1, 0)) *100 / sum(if(buy > 0, 1, 0)),2)  as p
from test.user_behavior_count"""
df=client.query_dataframe(sql)
df

### 用户行为转换率

In [ ]:
sql="""select a.pv,
       a.fav,
       a.cart,
       a.fav + a.cart                     `fav+cart`,
       a.buy,
       round((a.fav + a.cart) / a.pv, 4)  pv2favcart,
       round(a.buy / (a.fav + a.cart), 4) favcart2buy,
       round(a.buy / a.pv, 4)             pv2buy
from (select sum(pv)   pv,
             sum(fav)  fav,
             sum(cart) cart,
             sum(buy)  buy
      from test.user_behavior_count) a"""
client.query_dataframe(sql)

### 用户行为习惯

In [ ]:
sql="""select HOUR(timestamp)                  hour,
       sumIf(1, behavior_type = 'pv')   pv,
       sumIf(1, behavior_type = 'fav')  fav,
       sumIf(1, behavior_type = 'cart') cart,
       sumIf(1, behavior_type = 'buy') buy
from test.user_behavior
group by HOUR(timestamp)
order by hour"""
client.query_dataframe(sql)

### 一周用户的活跃分布

In [ ]:
sql="""select  toDayOfWeek(timestamp) weekday,
       sumIf(1,behavior_type='pv') pv,
       sumIf(1,behavior_type='fav') fav,
       sumIf(1,behavior_type='cart') cart,
       sumIf(1,behavior_type='buy') buy
from test.user_behavior
where DATE(timestamp) between '2017-11-27' and '2017-12-03'
group by weekday
order by weekday"""
client.query_dataframe(sql)

### 最近一次购买

In [ ]:
sql = """set allow_experimental_window_functions=1"""
client.execute(sql)
sql="""select user_id,
       dateDiff('day',max(timestamp),DATE('2017-12-04')) R,
       dense_rank() over( order by dateDiff('day',max(timestamp),DATE('2017-12-04'))) R_rank
from test.user_behavior
where behavior_type = 'buy'
group by user_id
limit 100;"""
client.query_dataframe(sql)

### 消费频率

In [ ]:
sql = """set allow_experimental_window_functions=1;"""
client.execute(sql)
sql="""select user_id,
       count() as F,
       dense_rank() over(order by count() desc) as F_rank
from test.user_behavior
where behavior_type = 'buy'
group by user_id
limit 10;"""
client.query_dataframe(sql)

In [ ]:
sql = """set allow_experimental_window_functions=1;"""
client.execute(sql)
sql="""select user_id,
       dateDiff('day', max(timestamp), DATE('2017-12-04')) R,
       dense_rank()                                        over( order by dateDiff('day',max(timestamp),DATE('2017-12-04'))) R_rank,
       count() as                                          F,
       dense_rank()                                        over(order by count() desc) as F_rank

from test.user_behavior
where behavior_type = 'buy'
group by user_id"""
client.query_dataframe(sql)

### 商品维度的分析

In [ ]:
sql="""select item_id,
       sumIf(1,behavior_type = 'pv')   as pv,
       sumIf(1,behavior_type = 'fav')  as fav,
       sumIf(1,behavior_type = 'cart') as cart,
       sumIf(1,behavior_type = 'buy')  as buy
from test.user_behavior
group by item_id
order by buy desc
limit 10"""
client.query_dataframe(sql)

In [ ]:
sql="""select category_id,
       sumIf(1,behavior_type = 'pv')   as pv,
       sumIf(1,behavior_type = 'fav')  as fav,
       sumIf(1,behavior_type = 'cart') as cart,
       sumIf(1,behavior_type = 'buy')  as buy
from test.user_behavior
group by category_id
order by buy desc
limit 10"""
client.query_dataframe(sql)